In [65]:
from base64 import decodebytes
from functools import lru_cache
from typing import Tuple

import k3d
import numpy as np
from pygltflib import (
    FLOAT,
    GLTF2,
    UNSIGNED_BYTE,
    UNSIGNED_INT,
    UNSIGNED_SHORT,
    Buffer,
    BufferFormat,
)

import awkward as ak

from matplotlib.cm import viridis

In [66]:
type_map = {UNSIGNED_BYTE: "B", UNSIGNED_SHORT: "u2", UNSIGNED_INT: "u4", FLOAT: "f4"}

component_map = {
    "SCALAR": 1,
    "VEC2": 2,
    "VEC3": 3,
    "VEC4": 4,
    "MAT2": 4,
    "MAT3": 9,
    "MAT4": 16,
}


def _gltf_uri_to_array(uri: str) -> np.ndarray:
    buffer_encoded = uri.split(",", 1)[1]
    buffer_string = decodebytes(buffer_encoded.encode())
    return np.frombuffer(buffer_string, dtype=np.uint8)


@lru_cache(512)
def _load_buffer_from_uri(uri: str) -> np.ndarray:
    return _gltf_uri_to_array(uri)


def load_file_as_embedded(filepath) -> GLTF2:
    gltf = GLTF2.load(filepath)
    gltf.convert_buffers(BufferFormat.DATAURI)
    return gltf


def _load_accessor(gltf: GLTF2, accessor) -> np.ndarray:
    view = gltf.bufferViews[accessor.bufferView]
    buffer = _load_buffer_from_uri(gltf.buffers[view.buffer].uri)
    data = buffer[view.byteOffset : view.byteOffset + view.byteLength]
    components = component_map[accessor.type]
    array = np.frombuffer(data, dtype=type_map[accessor.componentType])
    if components > 1:
        return array.reshape(-1, components)
    return array


def load_mesh(gltf, mesh) -> Tuple[np.ndarray, np.ndarray]:
    prim = mesh.primitives[0]
    positions = _load_accessor(gltf, gltf.accessors[prim.attributes.POSITION])
    indices = _load_accessor(gltf, gltf.accessors[prim.indices]).reshape(-1, 3)
    return positions, indices

In [67]:
gltf = load_file_as_embedded("/home/angus/Downloads/detector.gltf")

In [68]:
loaded = {mesh.name: load_mesh(gltf, mesh) for mesh in gltf.meshes}

In [69]:
good = [m for n, m in loaded.items() if not np.all(m[0][:, 2] < -200)]

In [70]:
v, i = zip(*good)

In [71]:
data = ak.zip({"vertex": v, "index": i}, depth_limit=1)

In [72]:
# si 1 (1, 17+4*6)
# cs 1 (41, 51)
# si 2 (51, 51+8*4)
# cs 2 (83,83+8)
# si 3 (91, 91+4*8)
# cs 3 (123, 123+8)
# si 4 (131, 131+8*4)
# cs 4 (163, 171)
# si 5 (171,171+4*8)
# si 6 (203, 203+8*4)
# cs 5 (235, 243)
# cs 6 (243, 251)

In [73]:
s_si = [np.s_[1:41], np.s_[51:83], np.s_[91:123], np.s_[131:163], np.s_[171:235]]
data_si = np.concatenate([data[s] for s in s_si])

In [74]:
offset_si = np.zeros(len(data_si.vertex))
offset_si[1:] = np.cumsum(ak.num(data_si.vertex, axis=1)[:-1])

silicon = ak.zip(
    {"vertex": data_si.vertex, "index": data_si.index + offset_si}, depth_limit=1
)

In [75]:
s_cs = [np.s_[41:51], np.s_[83:91], np.s_[121:131], np.s_[163:171], np.s_[235:251]]
data_cs = np.concatenate([data[s] for s in s_cs])

offset_cs = np.zeros(len(data_cs.vertex))
offset_cs[1:] = np.cumsum(ak.num(data_cs.vertex, axis=1)[:-1])

cesium = ak.zip(
    {"vertex": data_cs.vertex, "index": data_cs.index + offset_cs}, depth_limit=1
)

In [76]:
def rgb_to_packed(rgb):
    col = (np.clip(rgb, 0, 1) * 255).astype(np.uint8)
    return int(col[0] << 16 | col[1] << 8 | col[2])

In [ ]:
micromegas = ak.zip(
    {"vertex": data_cs.vertex, "index": data_cs.index + offset_cs}, depth_limit=1
)

In [77]:
col = viridis(np.linspace(0, 1, 3))

In [62]:
plot = k3d.plot()

plot += k3d.mesh(silicon.vertex, silicon.index, color=rgb_to_packed(col[0]), name="Si")
plot += k3d.mesh(cesium.vertex, cesium.index, color=rgb_to_packed(col[1]), name="CsI(Tl)")
plot += k3d.mesh(data[0].vertex * [[1, 1, -1]], data[0].index, color=rgb_to_packed(col[2]), name="MicroMegas")
plot.display()

Output()

In [48]:
ak.count(data[0].index)

36

In [73]:
len(v[51:83])

32

In [ ]:
v_si = ak.flatten

In [52]:
plot = k3d.plot()

for j, (v, i) in enumerate(good.values()):
    if j not in range(251, 260):
        continue
    plot += k3d.mesh(v, i)
plot.display()

Output()